In [1]:
import numpy as np
import pandas as pd
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [19]:
data

,y,x
0,0,4
1,0,1
2,0,9
3,1,0
4,1,2
...,...,...
4995,0,4
4996,0,0
4997,1,0
4998,1,3


In [20]:
x

array([[4],
       [1],
       [9],
       ...,
       [0],
       [3],
       [7]])

In [2]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [6]:
%%timeit
result_1=target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23 s per loop


In [32]:
pd.DataFrame(target_mean_v1(data, 'y', 'x'))

,0
0,0.469185
1,0.493617
2,0.498016
3,0.507099
4,0.468691
...,...
4995,0.469185
4996,0.509128
4997,0.507099
4998,0.476891


In [26]:
data.shape[0]

5000

In [4]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [28]:
data.loc[1, 'x']

1

In [23]:
%%timeit
result_2=target_mean_v2(data, 'y', 'x')


1 loop, best of 3: 258 ms per loop


In [25]:
target_mean_v2(data, 'y', 'x')

array([0.46918489, 0.49361702, 0.49801587, ..., 0.50709939, 0.47689076,
       0.52479339])

In [15]:
%load_ext cython

In [68]:
%%cython -a
import numpy as np 
cimport numpy as cnp
cpdef target_mean_v3(data, y_name, x_name):
    cdef int num = data.shape[0]
    cdef int[:] result = np.zeros[num]
    cdef int[:] value_dict_y = np.zeros[10]
    cdef int[:] count_dict_y = np.zeros[10]
    for i in range(num):
        index = data.loc[i, x_name]
        value = data.loc[i, y_name]
        if index not in value_dict_y:
            value_dict_y[index] = value
            count_dict_y[index] = 0
        else:
            value_dict_y[index] += value
            count_dict_y[index] += 1
    for i in range(num):
        index = data.loc[i, x_name]
        value = data.loc[i, y_name]
        result[i] = (value_dict_y[index] - value) / count_dict_y[index]
    return result

In [67]:
%timeit 
target_mean_v3(data, 'y', 'x')

TypeError: ignored